In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time
from PIL import Image, ImageDraw, ImageFont

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

In [4]:
def putText(source, x, y, text, scale=2.5, color=(255,255,255), fontFace = cv2.FONT_HERSHEY_SIMPLEX):
    org = (x,y)
    fontScale = scale
    thickness = 5
    lineType = cv2.LINE_AA
    cv2.putText(source, text, org, fontFace, fontScale, color, thickness, lineType)

In [5]:
def cv2AddChineseText(img, text, position, textColor=(0, 255, 0), textSize=30):
    if (isinstance(img, np.ndarray)):  # 判断是否OpenCV图片类型
        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    # 创建一个可以在给定图像上绘图的对象
    draw = ImageDraw.Draw(img)
    # 字体的格式
    fontStyle = ImageFont.truetype(
        "simsun.ttc", textSize, encoding="utf-8")
    # 绘制文本
    draw.text(position, text, textColor, font=fontStyle)
    # 转换回OpenCV格式
    return cv2.cvtColor(np.asarray(img), cv2.COLOR_RGB2BGR)

In [6]:
cap = cv2.VideoCapture(0)

Counter_LowerHand1 = 0
Counter_LowerHand2 = 0
Counter_TopHand = 0
Counter_Block = 0
Stage_Lower = None
Stage_Top = None
Stage_Block = None

a = 0

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # 右手角度
            RIGHT_SHOULDER = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            RIGHT_ELBOW = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            RIGHT_WRIST = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            # 左手角度
            LEFT_SHOULDER = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            LEFT_ELBOW = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            LEFT_WRIST = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            # 右肩角度
            LEFT_SHOULDER = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            RIGHT_SHOULDER = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            RIGHT_ELBOW = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]

            # 左肩角度
            RIGHT_SHOULDER = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            LEFT_SHOULDER = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            LEFT_ELBOW = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            # 右腳角度
            RIGHT_HIP = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            RIGHT_KNEE = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            
            RIGHT_ANKLE = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            #左腳角度
            LEFT_HIP = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            
            LEFT_KNEE = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            
            LEFT_ANKLE = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            
            # 右腳角度
            angleRightLeg = calculate_angle(RIGHT_HIP, RIGHT_KNEE, RIGHT_ANKLE)
            # 左腳角度
            angleLeftLeg = calculate_angle(LEFT_HIP, LEFT_KNEE, LEFT_ANKLE)
            # 右手角度
            angleRightArm = calculate_angle(RIGHT_SHOULDER, RIGHT_ELBOW, RIGHT_WRIST)
            # 左手角度
            angleLeftArm = calculate_angle(LEFT_SHOULDER, LEFT_ELBOW, LEFT_WRIST)
            # 右肩角度
            angleRightshoulder = calculate_angle(LEFT_SHOULDER, RIGHT_SHOULDER, RIGHT_ELBOW)
            # 左肩角度
            angleLeftshoulder = calculate_angle(RIGHT_SHOULDER, LEFT_SHOULDER, LEFT_ELBOW)
            
            
            # 顯示角度
            '''
            cv2.putText(image, str(angleRightArm),
                        tuple(np.multiply(RIGHT_ELBOW, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                        )
            cv2.putText(image, str(angleLeftArm),
                        tuple(np.multiply(LEFT_ELBOW, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                        )
            
            cv2.putText(image, str(angleRightshoulder),
                        tuple(np.multiply(RIGHT_SHOULDER, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                        )
            
            cv2.putText(image, str(angleLeftshoulder),
                        tuple(np.multiply(LEFT_SHOULDER, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                        )
            '''
            
            # 低手角度、肩膀、手肘的角度調整，如果在區間內，則開始計分
            if sec > 0 :
                if angleRightshoulder < 100 and angleRightshoulder > 65 and angleLeftshoulder < 100 and angleLeftshoulder > 65:
                    if angleRightArm < 180 and angleRightArm > 160 and angleLeftArm < 180 and angleLeftArm > 160 :
                        if angleRightLeg < 155:
                            Stage_Lower = "downRight"
                        if angleRightLeg > 170 and Stage_Lower =='downRight':
                            Stage_Lower="upRight"
                            Counter_LowerHand1 += 1
                        if angleLeftLeg < 155:
                            Stage_Lower = "downLeft"
                        if angleLeftLeg > 170 and Stage_Lower =='downLeft':
                            Stage_Lower="upLeft"
                            Counter_LowerHand2 += 1
            
            # 上手角度、肩膀、手肘的角度調整，如果在區間內，則開始計分
            if sec > 0 :
                if angleRightshoulder > 150 and angleRightshoulder < 180 and \
                        angleLeftshoulder > 150 and angleRightshoulder < 180 and \
                        angleRightArm > 40 and angleRightArm < 80 and \
                        angleLeftArm > 40 and angleLeftArm < 80:
                            Stage_Top = "TopDown"

            if sec > 0 :
                if angleRightshoulder > 100 and angleRightshoulder < 125 and \
                        angleLeftshoulder > 100 and angleRightshoulder < 125 and \
                        angleRightArm > 130 and angleLeftArm > 130 and Stage_Top == 'TopDown' :
                            Stage_Top = "TopUp"
                            Counter_TopHand += 1
                
            # 攔網角度、肩膀、手肘的角度調整，如果在區間內，則開始計分
            if sec > 0 :
                if angleRightshoulder > 90 and angleRightshoulder < 110 and \
                        angleLeftshoulder > 90 and angleRightshoulder < 110 and \
                        angleRightArm > 0 and angleRightArm < 20 and \
                        angleLeftArm > 0 and angleLeftArm < 20:
                            Stage_Block = "BlockDown"
            if sec > 0 :
                if angleRightshoulder > 90 and angleRightshoulder < 140 and \
                        angleLeftshoulder > 90 and angleRightshoulder < 140 and \
                        angleRightArm > 160 and angleLeftArm > 160 and Stage_Block == 'BlockDown' :
                            Stage_Block = "BlockUp"
                            Counter_Block += 1

        except:
            pass

        #cv2.circle(image, (0, 0), 120, (153, 204, 255), -1)

        #低手計分
        cv2.putText(image, str('Lower:'),
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        
        cv2.putText(image, str(Counter_LowerHand1 + Counter_LowerHand2),
                    (130, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        
        #高手計分
        cv2.putText(image, str('Top:'),
                    (10, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        
        cv2.putText(image, str(Counter_TopHand),
                    (130, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        
        #攔網計分
        cv2.putText(image, str('Block:'),
                    (10, 130),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        
        cv2.putText(image, str(Counter_Block),
                    (130, 130),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        
        #影像寬度
        w = int(1000)
        h = int(800)
        image = cv2.resize(image,(w,h))
        white = 255 - np.zeros((h,w,4), dtype='uint8')

        '''
        # 各支點連一條線
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )
        '''

        #cv2.imshow('Mediapipe Feed', image)

        key = cv2.waitKey(1)
        if key == 32:
            a = 1
            sec = 30  # 加入倒數秒數
            sec1 = sec + 3
            Counter_LowerHand1 = 0
            Counter_LowerHand2 = 0
            Counter_TopHand = 0
            Counter_Block = 0   
            
        elif key == ord('w'):
            Counter_LowerHand1 = 0
            Counter_LowerHand2 = 0
            Counter_TopHand = 0
            Counter_Block = 0   
            
        elif key == ord('q'):
            break
            
        if a == 0:
            output = image.copy()
            
        else:
            #遊戲倒數計時
            output = image.copy()  # 設定 output 和 photo 變數
            photo = image.copy()
            if sec > 6 :
                sec = sec - 0.04     # sec 不斷減少 0.04 ( 根據個人電腦效能設定，使其搭配 while 迴圈看起來像倒數一秒 )
                sec1 = sec1 - 0.04
                putText(output, 890, 80, str(int(sec)), 2.5, (0, 0, 0))  # 加入文字
            
            elif sec <= 6 and sec > 0 :
                sec = sec - 0.04     # sec 不斷減少 0.04 ( 根據個人電腦效能設定，使其搭配 while 迴圈看起來像倒數一秒 )
                sec1 = sec1 - 0.04
                putText(output, 890, 80, str(int(sec)), 2.5, (0, 0, 255))  # 加入文字
                #putText(output, 890, 150, str(int(sec1)), 2.5, (0, 0, 255))  # 加入文字
                
            elif sec1 <= 4 and sec1 > 0 :
                sec1 = sec1 - 0.04
                putText(output, 890, 80, str(int(sec)), 2.5, (0, 0, 255))  # 加入文字
                putText(output, 320, 400, str('Times up!!!'), 2.5, (0, 0, 255))  # 加入文字
            
            else :
                #cv2.circle(output, (320, 500), 120, (153, 204, 255), -1)
                #cv2.rectangle(output,(750,750),(300,450),(153, 204, 255),-1)
                cv2.rectangle(output,(750,750),(300,450),(224, 224, 224),-1)
                putText(output, 890, 80, str(int(sec)), 2.5, (0, 0, 255))  # 加入文字
                #putText(output, 320, 400, str('Times up!!!'), 2.5, (0, 0, 255))  # 加入文字
                
                if Counter_LowerHand1 + Counter_LowerHand2 >= 5 :
                    putText(output, 320, 520, str('Lower: Good'), 2, (0, 0, 255), )
                
                else :
                    putText(output, 320, 520, str('Lower: Bad'), 2, (0, 0, 255))
                    
                if Counter_TopHand >= 5 :
                    putText(output, 320, 620, str('Top: Good'), 2, (0, 0, 255))
                
                else :
                    putText(output, 320, 620, str('Top: Bad'), 2, (0, 0, 255))
                    
                if Counter_Block >= 5 :
                    putText(output, 320, 720, str('Block: Good'), 2, (0, 0, 255))
                    
                else :
                    putText(output, 320, 720, str('Block: Bad'),  2, (0, 0, 255))
                    
        
        cv2.imshow('Mediapipe Feed', output)

    cap.release()
    cv2.destroyAllWindows()